The code is based on the paper:
Towards Simulating Foggy and Hazy Images and Evaluating Their Authenticity.
Referenced Github: https://github.com/noahzn/FoHIS

The evaluator undergoes training utilizing images sourced from both the SHIFT dataset and the Dense Foggy dataset.

In [9]:
import cv2
import numpy as np
import os
import math
from PIL import Image
import scipy.io as sio
from scipy.special import gamma

In [10]:
BLOCK_SIZE_ROW     = 48
BLOCK_SIZE_COL     = 48
NORMALIZED_WIDTH   = 528
FEATURE_NUMBER     = 16
GRADIENT_THRESHOLD_L = 3
GRADIENT_THRESHOLD_R = 60
DARK_CHANNEL_THRESHOLD_L = 30
DARK_CHANNEL_THRESHOLD_R = 100

In [11]:
def filter2d(input_img, filter, frame):
    size = len(input_img), len(input_img[0])
    output = []
    for i in range(size[0]):
        temp = []
        for j in range(size[1]):
            temp.append(filter(input_img, (i, j), frame))

        output.append(temp)
    return output

In [12]:
def getDark(input_img, filter, frame):
    size = input_img.size
    output = []

    for x in range(size[1]):
        temp = []
        for y in range(size[0]):
            temp.append(min(input_img.getpixel((y, x))))

        output.append(temp)

    output = filter2d(output, filter, frame)

    output_img = Image.new('L', size)

    for x in range(size[1]):
        for y in range(size[0]):
            output_img.putpixel((y, x), output[x][y])

    return output_img

In [13]:
def minimizeFilter(input_img, point, size):
    begin = (point[0] - size[0] / 2, point[0] + size[0] / 2 + 1)
    end = (point[1] - size[1] / 2, point[1] + size[1] / 2 + 1)
    begin1, begin2 = int(begin[0]), int(begin[1])
    end1, end2 = int(end[0]), int(end[1])
    l = []
    for i in range(begin1, begin2):
        for j in range(end1, end2):
            if (i >= 0 and i < len(input_img)) and (j >= 0 and j < len(input_img[0])):
                l.append(input_img[i][j])
    return min(l)

In [14]:
def estimate_aggd_parameter(vec):
    vec = np.nan_to_num(vec)
    # print(vec)
    gam = [x/1000 for x in range(200, 10001, 1)]
    r_gam = [(gamma(2/x)**2/(gamma(1/x)*gamma(3/x))) for x in gam]
    leftstd = np.nan_to_num(np.sqrt(np.mean(vec[vec < 0]**2)))
    rightstd = np.nan_to_num(np.sqrt(np.mean(vec[vec > 0]**2)))
    gammahat = np.nan_to_num(leftstd / (rightstd+0.00001))
    rhat = np.nan_to_num((np.mean(np.abs(vec))**2) / np.nanmean(vec**2))
    rhatnorm = (rhat*(gammahat**3 + 1)*(gammahat + 1))/((gammahat**2 + 1)**2)
    m1 = (r_gam - rhatnorm)**2
    m2 = m1.tolist()
    array_position = m2.index(np.min(m1))
    alpha = gam[array_position]
    beta_l = leftstd * np.sqrt(gamma(1/alpha)/gamma(3/alpha))
    beta_r = rightstd * np.sqrt(gamma(1/alpha)/gamma(3/alpha))
    return alpha, beta_l, beta_r

In [15]:
def compute_features(gray, gradient):
    features = []
    for m in [gray]:
        μ = cv2.GaussianBlur(m, (5, 5), 5/6, borderType=cv2.BORDER_REPLICATE)
        σ = np.sqrt(abs(cv2.GaussianBlur(m*m, (5, 5), 5/6, borderType=cv2.BORDER_REPLICATE) - μ*μ))
        I = (m - μ) / (σ + 1)
        alpha, beta_l, beta_r = estimate_aggd_parameter(I)
        features.append(alpha)
        features.append((beta_l+beta_r)/2)
        I = np.log(I + 0.00001)
        shift1 = [(0, 1), (1, 0), (1, 1), (1, -1)]
        shift2 = [(-1, 0), (1, 0), (0, -1), (0, 1), (0, 0), (1, 1), (0, 1), (1, 0), (-1, -1), (1, 1), (-1, 1), (1, -1)]
        for i in shift1:
            D = np.roll(I, i, axis=(0, 1)) - I
            alpha, beta_l, beta_r = estimate_aggd_parameter(D)
            features.append(alpha)
            features.append((beta_l+beta_r)/2)

        for i in range(3):
            D = np.roll(I, shift2[4*i], axis=(0, 1)) + np.roll(I, shift2[4*i+1], axis=(0, 1)) \
                - np.roll(I, shift2[4*i+2], axis=(0, 1)) - np.roll(I, shift2[4*i+3], axis=(0, 1))
            alpha, beta_l, beta_r = estimate_aggd_parameter(D)

            features.append(alpha)
            features.append((beta_l+beta_r)/2)

    return features

In [16]:
features1_list_all = []
features2_list_all = []

for root, dirs, files in os.walk('/Users/abhinandp/PycharmProjects/3DCV-Project/3DCV-Project/Foggy_Images'):
    for fn in files:
        file_path = os.path.join(root, fn)
        img = cv2.imread(file_path)

        if img is None:
            print(f"Failed to load image: {file_path}")
            continue

        # Resize the image
        img_resized = cv2.resize(img, (NORMALIZED_WIDTH, NORMALIZED_WIDTH), interpolation=cv2.INTER_CUBIC)

        for iters in range(1):
            print(fn, iters)
            img_resized = cv2.resize(img_resized, (int(NORMALIZED_WIDTH / (iters + 1)), int(NORMALIZED_WIDTH / (iters + 1))), interpolation=cv2.INTER_CUBIC)
            gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)

            BLOCK_SIZE_COL = int(BLOCK_SIZE_COL / (iters + 1))
            BLOCK_SIZE_ROW = int(BLOCK_SIZE_ROW / (iters + 1))

            block_rownum = math.floor(gray.shape[0] / BLOCK_SIZE_ROW)
            block_colnum = math.floor(gray.shape[1] / BLOCK_SIZE_COL)
            img_resized = img_resized[:block_rownum * BLOCK_SIZE_ROW, :block_colnum * BLOCK_SIZE_COL, :]
            gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)

            gradx = cv2.Sobel(gray, cv2.CV_16S, 1, 0, ksize=3)
            grady = cv2.Sobel(gray, cv2.CV_16S, 0, 1, ksize=3)
            absX = cv2.convertScaleAbs(gradx)
            absY = cv2.convertScaleAbs(grady)
            gradient = cv2.addWeighted(absX, 0.5, absY, 0.5, 0)
            gradient2 = gradient
            gradient2[gradient2 < 20] = 0
            gradient2[gradient2 >= 20] = 255

            dark_image = np.asarray(getDark(Image.fromarray(np.uint8(img_resized)), minimizeFilter, (int(10 / (iters + 1)), int(10 / (iters + 1)))))

            for i in range(block_rownum):
                for j in range(block_colnum):
                    features1_list = []
                    features2_list = []
                    crop_row_start = i * BLOCK_SIZE_ROW
                    crop_row_end = (i + 1) * BLOCK_SIZE_ROW
                    crop_col_start = j * BLOCK_SIZE_COL
                    crop_col_end = (j + 1) * BLOCK_SIZE_COL

                    crop_gray = gray[crop_row_start: crop_row_end, crop_col_start:crop_col_end]
                    crop_gradient = gradient[crop_row_start: crop_row_end, crop_col_start:crop_col_end]
                    crop_gradient2 = gradient2[crop_row_start: crop_row_end, crop_col_start:crop_col_end]
                    crop_dark_image = dark_image[crop_row_start: crop_row_end, crop_col_start:crop_col_end]

                    if np.mean(crop_dark_image) < DARK_CHANNEL_THRESHOLD_L:
                        if np.count_nonzero(crop_gradient2) > 400:
                            features1_list.extend(compute_features(crop_gray.astype(np.float64), crop_gradient.astype(np.float64)))
                            cv2.rectangle(img_resized, (crop_col_start, crop_row_start), (crop_col_end, crop_row_end), (0, 255, 0))
                        else:
                            features1_list.extend(compute_features(crop_gray.astype(np.float64), crop_gradient.astype(np.float64)))
                            cv2.rectangle(img_resized, (crop_col_start, crop_row_start), (crop_col_end, crop_row_end), (255, 0, 0))

                    elif np.mean(crop_dark_image) > DARK_CHANNEL_THRESHOLD_R:
                        if np.count_nonzero(crop_gradient2) < 80:
                            features2_list.extend(compute_features(crop_gray.astype(np.float64), crop_gradient.astype(np.float64)))
                            cv2.rectangle(img_resized, (crop_col_start, crop_row_start), (crop_col_end, crop_row_end), (255, 0, 255))

                    features1_list_all.extend(features1_list)
                    features2_list_all.extend(features2_list)

            print(len(features1_list_all), len(features2_list_all))
            # cv2.imshow('img', img_resized)
            # cv2.waitKey()
            cv2.imwrite(os.path.join('/Users/abhinandp/PycharmProjects/3DCV-Project/3DCV-Project/Foggy-Evaluation', fn.replace('.jpg', '_result_d.jpg')), img_resized)

2018-12-16_12-31-35_00000.png 0


/var/folders/vw/c2j4rkjd52z6m0m84r33nr7r0000gn/T/ipykernel_12655/22838383.py:10: RuntimeWarning: invalid value encountered in log
  I = np.log(I + 0.00001)


0 176
2018-10-29_14-30-29_02510.png 0
64 512
2018-10-29_15-02-37_01830.png 0
208 576
2018-12-12_09-34-31_00600.png 0
208 1440
2018-10-08_08-10-40_01710.png 0
352 1552
2018-12-12_11-27-36_00000.png 0
480 1568
2018-10-29_14-30-29_01780.png 0
560 1760
2018-03-15_09-35-28_00500.png 0
640 1776
2018-10-29_15-02-37_00910.png 0
752 2016
2018-10-08_08-50-46_00150.png 0
832 2368
2018-10-29_16-18-47_01700.png 0
832 2592
2018-10-29_15-25-20_01990.png 0
864 2768
2018-12-12_09-22-59_05400.png 0
864 3168
2018-10-29_15-46-53_02600.png 0
960 3776
2018-10-08_08-27-03_01760.png 0
1168 4224
2018-10-08_08-10-40_00630.png 0
1248 4224
2018-10-29_15-53-13_03990.png 0
1376 4528
2018-10-29_14-35-02_01500.png 0
1488 4576
2018-02-04_12-42-39_00000.png 0
1616 4800
2018-12-12_13-43-01_00100.png 0
1648 4896
2018-10-29_14-30-29_02100.png 0
1728 5024
2018-10-08_08-50-46_08450.png 0
1808 5344
2018-10-08_08-18-59_02500.png 0
1920 5632
2018-10-29_14-50-14_00300.png 0
2016 5840
2018-10-29_15-37-43_01400.png 0
2112 6224
20

/var/folders/vw/c2j4rkjd52z6m0m84r33nr7r0000gn/T/ipykernel_12655/2329417900.py:9: RuntimeWarning: invalid value encountered in scalar divide
  rhat = np.nan_to_num((np.mean(np.abs(vec))**2) / np.nanmean(vec**2))


26608 74528
2018-10-08_08-27-03_01200.png 0
26720 74960
2018-10-08_08-27-03_05700.png 0
26832 75440
2018-10-29_14-30-29_01790.png 0
26896 75680
2018-10-29_14-30-29_02500.png 0
26944 76000
2018-10-08_08-18-59_02100.png 0
27040 76448
2018-10-29_14-50-14_00700.png 0
27152 76768
2018-10-29_15-02-37_01820.png 0
27264 76848
2018-03-15_09-30-35_00000.png 0
27312 77264
2018-12-20_12-02-06_01500.png 0
27312 77344
2018-10-08_08-50-46_07310.png 0
27456 77568
2018-10-29_14-30-29_01720.png 0
27536 77696
2018-10-08_08-10-40_03170.png 0
27664 77952
2019-01-09_14-54-03_01000.png 0
27744 78064
2018-10-29_15-02-37_00230.png 0
27856 78128
2018-10-08_08-10-40_02520.png 0
28096 78240
2018-12-21_10-52-52_02900.png 0
28224 78624
2018-10-29_15-42-40_01500.png 0
28304 78800
2018-10-29_14-50-14_03050.png 0
28368 78816
2018-10-29_14-30-29_00600.png 0
28432 79104
2018-10-08_08-18-59_04700.png 0
28560 79472
2018-10-29_14-30-29_01250.png 0
28720 79792
2018-10-29_14-50-14_02400.png 0
28864 79984
2018-10-08_08-50-46_

In [17]:
 features1 = np.array(features1_list_all).reshape((int(len(features1_list_all)/FEATURE_NUMBER)), FEATURE_NUMBER)
features2 = np.array(features2_list_all).reshape((int(len(features2_list_all)/FEATURE_NUMBER)), FEATURE_NUMBER)
mu_prisparam1 = (np.mean(features1, axis=0))
mu_prisparam2 = (np.mean(features2, axis=0))
cov_prisparam1 = np.cov(features1.reshape(features1.shape[1], features1.shape[0]))
cov_prisparam2 = np.cov(features2.reshape(features2.shape[1], features2.shape[0]))
print(features2.shape, cov_prisparam2.shape)
sio.savemat('Shift&Dense-Dataset-1.mat', {'mu1': mu_prisparam1, 'mu2': mu_prisparam2, 'cov1': cov_prisparam1, 'cov2': cov_prisparam2})

(19090, 16) (16, 16)
